In [1]:
%load_ext autoreload

# Auto reloading causes the kernel to reload the libraries we have
%autoreload 2

import requests


In [88]:
# Get the current status
resp = requests.get('http://localhost:8089/stats/requests')
resp.json()

{'current_response_time_percentile_50': 26.285171508789062,
 'current_response_time_percentile_95': 110,
 'errors': [],
 'fail_ratio': 0.0,
 'state': 'running',
 'stats': [{'avg_content_length': 4.0,
   'avg_response_time': 50.089660294016106,
   'current_rps': 7.5,
   'max_response_time': 405.712366104126,
   'median_response_time': 27.404308319091797,
   'method': 'GET',
   'min_response_time': 23.899078369140625,
   'name': '/function/pyfibo/',
   'num_failures': 0,
   'num_requests': 838},
  {'avg_content_length': 4.0,
   'avg_response_time': 50.089660294016106,
   'current_rps': 7.5,
   'max_response_time': 405.712366104126,
   'median_response_time': 27.404308319091797,
   'method': None,
   'min_response_time': 23.899078369140625,
   'name': 'Total',
   'num_failures': 0,
   'num_requests': 838}],
 'total_rps': 7.5,
 'user_count': 50}

In [102]:
# set the target state
resp = requests.post('http://localhost:8089/swarm', data={'locust_count':50, 'hatch_rate': 10})
resp.json()

{'message': 'Swarming started', 'success': True}

In [101]:
# Stop the test
resp = requests.get('http://localhost:8089/stop')
resp.json()

{'message': 'Test stopped', 'success': True}

In [104]:
# reset stats
resp = requests.get('http://localhost:8089/stats/reset')
resp.text

'ok'

In [105]:
# Low-level stats

def get_current_stats(base='http://localhost:8089/'):
    resp = requests.get(base + 'stats/requests')
    return resp.json()

def set_target_user_count(locust_count, hatch_rate=50, base='http://localhost:8089/'):
    resp = requests.post(base+'swarm', data={'locust_count':locust_count, 'hatch_rate': hatch_rate})
    try:
        resp_json = resp.json()
        return resp_json['success']
    except:
        return False
    
def stop_test(base='http://localhost:8089/'):
    resp = requests.get(base+'stop')
    try:
        resp_json = resp.json()
        return resp_json['success']
    except:
        return False
    
def reset_stats(base='http://localhost:8089/'):
    resp = requests.get(base+'stats/reset')
    if resp.text == 'ok':
        return True
    else:
        return False

In [106]:
get_current_stats()

{'current_response_time_percentile_50': None,
 'current_response_time_percentile_95': None,
 'errors': [],
 'fail_ratio': 0.0,
 'state': 'stopped',
 'stats': [{'avg_content_length': 4.0,
   'avg_response_time': 45.997609416578314,
   'current_rps': 7.4,
   'max_response_time': 140.98405838012695,
   'median_response_time': 27.135848999023438,
   'method': 'GET',
   'min_response_time': 23.676395416259766,
   'name': '/function/pyfibo/',
   'num_failures': 0,
   'num_requests': 607},
  {'avg_content_length': 4.0,
   'avg_response_time': 45.997609416578314,
   'current_rps': 7.4,
   'max_response_time': 140.98405838012695,
   'median_response_time': 27.135848999023438,
   'method': None,
   'min_response_time': 23.676395416259766,
   'name': 'Total',
   'num_failures': 0,
   'num_requests': 607}],
 'total_rps': 7.4,
 'user_count': 0}

In [108]:
set_target_user_count(50)

True

In [111]:
reset_stats()

True

In [112]:
stop_test()

True

In [113]:
get_current_stats()

{'current_response_time_percentile_50': 26.113033294677734,
 'current_response_time_percentile_95': 120,
 'errors': [],
 'fail_ratio': 0.0,
 'state': 'stopped',
 'stats': [{'avg_content_length': 4.0,
   'avg_response_time': 38.28223461800433,
   'current_rps': 7.2,
   'max_response_time': 124.89724159240723,
   'median_response_time': 25.9859561920166,
   'method': 'GET',
   'min_response_time': 24.09195899963379,
   'name': '/function/pyfibo/',
   'num_failures': 0,
   'num_requests': 94},
  {'avg_content_length': 4.0,
   'avg_response_time': 38.28223461800433,
   'current_rps': 7.2,
   'max_response_time': 124.89724159240723,
   'median_response_time': 25.9859561920166,
   'method': None,
   'min_response_time': 24.09195899963379,
   'name': 'Total',
   'num_failures': 0,
   'num_requests': 94}],
 'total_rps': 7.2,
 'user_count': 0}